In [30]:
import numpy as np #thư viện tính toán
import pandas as pd #thư viện xử lý dữ liệu
import matplotlib.pyplot as plt #thư viện vẽ đồ thị
import os #thư viện hệ thống
import cv2 #thư viện xử lý ảnh

In [31]:
df = pd.read_csv('labels_imgs.csv') #đọc file csv chứa các nhãn
df.head() #hiển thị 5 dòng đầu tiên

,filepath,xmin,xmax,ymin,ymax
0,./imgs\10148.xml,158,245,159,180
1,./imgs\10149.xml,234,313,129,153
2,./imgs\10690.xml,144,234,181,203
3,./imgs\10691.xml,220,307,149,171
4,./imgs\1070.xml,86,187,110,133


In [32]:
import xml.etree.ElementTree as xet #thư viện xử lý file xml

In [33]:
filename = df['filepath'][4] #lấy tên file ảnh đầu tiên
filename #hiển thị tên file

'./imgs\\1070.xml'

In [35]:
#Hàm getFileName trả về tên file ảnh
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text #lấy tên file ảnh
    filepath_image = os.path.join('./imgs',filename_image) #tạo đường dẫn đến file ảnh
    return filepath_image #trả về tên file ảnh

In [36]:
getFilename(filename) #gọi hàm getFilename lấy tên file ảnh đầu tiên

'./imgs\\1070.jpg'

In [37]:
image_path = list(df['filepath'].apply(getFilename)) #lấy tên file ảnh từ tập dữ liệu , .apply() là hàm áp dụng hàm getFilename cho tất cả các dòng trong tập dữ liệu
image_path 

['./imgs\\10148.jpg',
 './imgs\\10149.jpg',
 './imgs\\10690.jpg',
 './imgs\\10691.jpg',
 './imgs\\1070.jpg',
 './imgs\\1071.jpg',
 './imgs\\10997.jpg',
 './imgs\\11024.jpg',
 './imgs\\11025.jpg',
 './imgs\\11044.jpg',
 './imgs\\11045.jpg',
 './imgs\\11084.jpg',
 './imgs\\11090.jpg',
 './imgs\\11118.jpg',
 './imgs\\11119.jpg',
 './imgs\\11172.jpg',
 './imgs\\11173.jpg',
 './imgs\\11210.jpg',
 './imgs\\11211.jpg',
 './imgs\\11276.jpg',
 './imgs\\11278.jpg',
 './imgs\\11279.jpg',
 './imgs\\11292.jpg',
 './imgs\\11473.jpg',
 './imgs\\11477.jpg',
 './imgs\\11508.jpg',
 './imgs\\11509.jpg',
 './imgs\\11564.jpg',
 './imgs\\11565.jpg',
 './imgs\\11664.jpg',
 './imgs\\11665.jpg',
 './imgs\\11842.jpg',
 './imgs\\11843.jpg',
 './imgs\\12214.jpg',
 './imgs\\12215.jpg',
 './imgs\\12244.jpg',
 './imgs\\12245.jpg',
 './imgs\\12264.jpg',
 './imgs\\12265.jpg',
 './imgs\\12276.jpg',
 './imgs\\12277.jpg',
 './imgs\\12304.jpg',
 './imgs\\12305.jpg',
 './imgs\\12310.jpg',
 './imgs\\12311.jpg',
 './imgs\\12

#### verify image and output (hiển thị ảnh đầu vào và vẽ bounding box lên ảnh đó)

In [47]:
file_path = image_path[0]
file_path 

'./imgs\\10148.jpg'

In [48]:
img = cv2.imread(file_path) #đọc file ảnh

cv2.namedWindow('example',cv2.WINDOW_NORMAL) #tạo cửa sổ có tên là example
cv2.imshow('example',img) #hiển thị ảnh trong cửa sổ example
cv2.waitKey(0) #đợi người dùng nhấn phím bất kì
cv2.destroyAllWindows() #đóng cửa sổ

In [49]:
# 1093	1396	645	727
cv2.rectangle(img,(158,159),(245,180),(0,255,0),3) #vẽ hình chữ nhật bounding box trên ảnh, (0,255,0) là màu xanh lá cây, 3 là độ dày của đường viền, (1093,645) là tọa độ góc trái trên, (1396,727) là tọa độ góc phải dưới
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Data Preprocessing (tiền xử lý dữ liệu)

In [50]:
from sklearn.model_selection import train_test_split #thư viện chia tập dữ liệu thành tập train và tập test
from tensorflow.keras.preprocessing.image import load_img, img_to_array #thư viện xử lý ảnh, load_img() để đọc ảnh, img_to_array() để chuyển ảnh thành mảng numpy

In [51]:
labels = df.iloc[:,1:].values #lấy tất cả các cột từ cột thứ 2 trở đi (lấy xmin, xmax, ymin, ymax)

In [52]:
data = []  #khởi tạo mảng data lưu trữ ảnh xám đã được chuyển thành mảng numpy
output = [] #khởi tạo mảng output lưu trữ các nhãn (vị trí bounding box) đã điều chỉnh sau khi resize ảnh
for ind in range(len(image_path)): #duyệt qua tất cả các ảnh
    image = image_path[ind] #lấy tên file ảnh
    img_arr = cv2.imread(image) #đọc ảnh
    h,w,d = img_arr.shape #lấy chiều cao, chiều rộng, số kênh màu của ảnh
    # prepprocesing tiền xử lý ảnh
    load_image = load_img(image,target_size=(224,224)) #đọc ảnh, resize ảnh về kích thước 224x224 (vì mô hình YOLOv3 chỉ nhận ảnh có kích thước 224x224)
    load_image_arr = img_to_array(load_image) #chuyển ảnh thành mảng numpy
    norm_load_image_arr = load_image_arr/255.0 # normalization ảnh, đưa các giá trị trong mảng về khoảng 0-1
    # normalization to labels, tính toán lại các giá trị của bounding box theo tỉ lệ của ảnh
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w #tính toán lại tọa độ bounding box sau khi resize ảnh
    nymin,nymax = ymin/h,ymax/h #tính toán lại tọa độ bounding box sau khi resize ảnh
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output, tọa độ bounding box sau khi resize ảnh
    # -------------- append
    data.append(norm_load_image_arr) #thêm ảnh đã được xử lý thành ảnh xám vào mảng data
    output.append(label_norm) #thêm tọa độ bounding box đã được xử lý vào mảng output

In [53]:
X = np.array(data,dtype=np.float32) #chuyển mảng data thành mảng numpy
y = np.array(output,dtype=np.float32) #chuyển mảng output thành mảng numpy

In [54]:
X.shape,y.shape #hình dạng của mảng X và y, X có 225 ảnh, mỗi ảnh có kích thước 224x224x3, y có 225 ảnh, mỗi ảnh có 4 giá trị tọa độ bounding box

((1218, 224, 224, 3), (1218, 4))

In [55]:
#chia tập dữ liệu thành tập train và tập test
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0) #chia tập dữ liệu thành tập train và tập test, tập train chiếm 80% dữ liệu, tập test chiếm 20% dữ liệu, random_state=0 để tạo ra cùng một tập dữ liệu khi chạy lại code
x_train.shape,x_test.shape,y_train.shape,y_test.shape #hình dạng của tập train và tập test sau khi chia, tập train có 180 ảnh, tập test có 45 ảnh

((974, 224, 224, 3), (244, 224, 224, 3), (974, 4), (244, 4))

### Deep Learning Model (Mô hình học sâu)

In [56]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2 #import các mô hình MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input #import các lớp Dense, Dropout, Flatten, Input để xây dựng mô hình
from tensorflow.keras.models import Model #import Model để xây dựng mô hình
import tensorflow as tf #import tensorflow

In [57]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3))) #khởi tạo mô hình InceptionResNetV2, weights="imagenet" để load các trọng số của mô hình đã được huấn luyện trên ImageNet, include_top=False để không sử dụng lớp output của mô hình, input_tensor=Input(shape=(224,224,3)) để chỉ định kích thước của ảnh đầu vào
inception_resnet.trainable=False #đóng băng các trọng số của mô hình InceptionResNetV2
# ---------------------
headmodel = inception_resnet.output #lấy output của mô hình InceptionResNetV2
headmodel = Flatten()(headmodel) #chuyển output của mô hình InceptionResNetV2 thành mảng 1 chiều
headmodel = Dense(500,activation="relu")(headmodel) #thêm lớp Dense với 500 nút ẩn và hàm kích hoạt relu
headmodel = Dense(250,activation="relu")(headmodel) #thêm lớp Dense với 250 nút ẩn và hàm kích hoạt relu
headmodel = Dense(4,activation='sigmoid')(headmodel) #thêm lớp Dense với 4 nút ẩn và hàm kích hoạt sigmoid
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel) #xây dựng mô hình bằng cách kết nối input của mô hình InceptionResNetV2 với output của headmodel

In [58]:
# complie model: biên dịch mô hình
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics = ["accuracy", tf.metrics.AUC()]) #sử dụng hàm mất mát MSE, sử dụng thuật toán tối ưu Adam với learning_rate=1e-4
model.summary() #xem tóm tắt mô hình

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_203 (Conv2D)            (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_203 (Batch  (None, 111, 111, 32  96         ['conv2d_203[0][0]']             
 Normalization)                 )                                                           

 batch_normalization_208 (Batch  (None, 25, 25, 96)  288         ['conv2d_208[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_210 (Batch  (None, 25, 25, 64)  192         ['conv2d_210[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_213 (Batch  (None, 25, 25, 96)  288         ['conv2d_213[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_214 (Batch  (None, 25, 25, 64)  192         ['conv2d_214[0][0]']             
 Normaliza

 activation_224 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_224[0][0]']
                                                                                                  
 conv2d_222 (Conv2D)            (None, 25, 25, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_225 (Conv2D)            (None, 25, 25, 48)   13824       ['activation_224[0][0]']         
                                                                                                  
 batch_normalization_222 (Batch  (None, 25, 25, 32)  96          ['conv2d_222[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_225 (Batch  (None, 25, 25, 48)  144         ['conv2d_225[0][0]']             
 Normaliza

                                                                                                  
 activation_227 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_227[0][0]']
                                                                                                  
 activation_229 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_229[0][0]']
                                                                                                  
 activation_232 (Activation)    (None, 25, 25, 64)   0           ['batch_normalization_232[0][0]']
                                                                                                  
 block35_3_mixed (Concatenate)  (None, 25, 25, 128)  0           ['activation_227[0][0]',         
                                                                  'activation_229[0][0]',         
                                                                  'activation_232[0][0]']         
          

 Normalization)                                                                                   
                                                                                                  
 batch_normalization_243 (Batch  (None, 25, 25, 48)  144         ['conv2d_243[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_240 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_240[0][0]']
                                                                                                  
 activation_243 (Activation)    (None, 25, 25, 48)   0           ['batch_normalization_243[0][0]']
                                                                                                  
 conv2d_239 (Conv2D)            (None, 25, 25, 32)   10240       ['block35_4_ac[0][0]']           
          

...

                                                                                                  
 activation_372 (Activation)    (None, 5, 5, 224)    0           ['batch_normalization_372[0][0]']
                                                                                                  
 conv2d_370 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_1_ac[0][0]']            
                                                                                                  
 conv2d_373 (Conv2D)            (None, 5, 5, 256)    172032      ['activation_372[0][0]']         
                                                                                                  
 batch_normalization_370 (Batch  (None, 5, 5, 192)   576         ['conv2d_370[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_nor

 batch_normalization_378 (Batch  (None, 5, 5, 192)   576         ['conv2d_378[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_381 (Batch  (None, 5, 5, 256)   768         ['conv2d_381[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_378 (Activation)    (None, 5, 5, 192)    0           ['batch_normalization_378[0][0]']
                                                                                                  
 activation_381 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_381[0][0]']
                                                                                                  
 block8_4_

                                                                                                  
 activation_389 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_389[0][0]']
                                                                                                  
 block8_6_mixed (Concatenate)   (None, 5, 5, 448)    0           ['activation_386[0][0]',         
                                                                  'activation_389[0][0]']         
                                                                                                  
 block8_6_conv (Conv2D)         (None, 5, 5, 2080)   933920      ['block8_6_mixed[0][0]']         
                                                                                                  
 block8_6 (Lambda)              (None, 5, 5, 2080)   0           ['block8_5_ac[0][0]',            
                                                                  'block8_6_conv[0][0]']          
          

                                                                                                  
 block8_8 (Lambda)              (None, 5, 5, 2080)   0           ['block8_7_ac[0][0]',            
                                                                  'block8_8_conv[0][0]']          
                                                                                                  
 block8_8_ac (Activation)       (None, 5, 5, 2080)   0           ['block8_8[0][0]']               
                                                                                                  
 conv2d_399 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_8_ac[0][0]']            
                                                                                                  
 batch_normalization_399 (Batch  (None, 5, 5, 192)   576         ['conv2d_399[0][0]']             
 Normalization)                                                                                   
          

 )                                                                                                
                                                                                                  
 conv_7b_ac (Activation)        (None, 5, 5, 1536)   0           ['conv_7b_bn[0][0]']             
                                                                                                  
 flatten_1 (Flatten)            (None, 38400)        0           ['conv_7b_ac[0][0]']             
                                                                                                  
 dense_3 (Dense)                (None, 500)          19200500    ['flatten_1[0][0]']              
                                                                                                  
 dense_4 (Dense)                (None, 250)          125250      ['dense_3[0][0]']                
                                                                                                  
 dense_5 (

### model training

In [59]:
from tensorflow.keras.callbacks import TensorBoard #import TensorBoard để lưu lại quá trình huấn luyện mô hình

In [60]:
tfb = TensorBoard('object_detection') #khởi tạo TensorBoard, 'object_detection' là tên thư mục lưu lại quá trình huấn luyện mô hình

In [ ]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb]) #tiến hành huấn luyện mô hình, batch_size=10 là số lượng ảnh trong mỗi batch (tùy thuộc vào tốc độ máy tính), epochs=100 là số lần lặp lại quá trình huấn luyện, validation_data=(x_test,y_test) để kiểm tra độ chính xác của mô hình trên tập test sau mỗi lần lặp lại quá trình huấn luyện, callbacks=[tfb] để lưu lại quá trình huấn luyện mô hình

Epoch 1/100
98/98 [==============================] - 100s 945ms/step - loss: 0.1668 - accuracy: 0.4168 - auc_1: 0.0000e+00 - val_loss: 0.1609 - val_accuracy: 0.4385 - val_auc_1: 0.0000e+00
Epoch 2/100
98/98 [==============================] - 98s 1s/step - loss: 0.1669 - accuracy: 0.4148 - auc_1: 0.0000e+00 - val_loss: 0.1609 - val_accuracy: 0.4385 - val_auc_1: 0.0000e+00
Epoch 3/100
98/98 [==============================] - 93s 951ms/step - loss: 0.1669 - accuracy: 0.4148 - auc_1: 0.0000e+00 - val_loss: 0.1609 - val_accuracy: 0.4385 - val_auc_1: 0.0000e+00
Epoch 4/100
98/98 [==============================] - 96s 981ms/step - loss: 0.1669 - accuracy: 0.4148 - auc_1: 0.0000e+00 - val_loss: 0.1609 - val_accuracy: 0.4385 - val_auc_1: 0.0000e+00
Epoch 5/100
98/98 [==============================] - 92s 942ms/step - loss: 0.1669 - accuracy: 0.4148 - auc_1: 0.0000e+00 - val_loss: 0.1609 - val_accuracy: 0.4385 - val_auc_1: 0.0000e+00
Epoch 6/100
98/98 [==============================] - 94s 965ms

In [29]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,
                    validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=100) #tiến hành huấn luyện mô hình, batch_size=10 là số lượng ảnh trong mỗi batch, epochs=200 là số lần lặp lại quá trình huấn luyện, validation_data=(x_test,y_test) để kiểm tra độ chính xác của mô hình trên tập test sau mỗi lần lặp lại quá trình huấn luyện, callbacks=[tfb] để lưu lại quá trình huấn luyện mô hình, initial_epoch=101 để tiếp tục huấn luyện mô hình từ epoch 101

Epoch 101/200
7/7 [==============================] - 6s 870ms/step - loss: 0.0676 - accuracy: 0.5625 - auc: 0.0000e+00 - val_loss: 0.0569 - val_accuracy: 0.5882 - val_auc: 0.0000e+00
Epoch 102/200
7/7 [==============================] - 6s 877ms/step - loss: 0.0674 - accuracy: 0.5469 - auc: 0.0000e+00 - val_loss: 0.0574 - val_accuracy: 0.5882 - val_auc: 0.0000e+00
Epoch 103/200
7/7 [==============================] - 6s 870ms/step - loss: 0.0675 - accuracy: 0.5625 - auc: 0.0000e+00 - val_loss: 0.0572 - val_accuracy: 0.5882 - val_auc: 0.0000e+00
Epoch 104/200
7/7 [==============================] - 6s 884ms/step - loss: 0.0681 - accuracy: 0.5312 - auc: 0.0000e+00 - val_loss: 0.0569 - val_accuracy: 0.5882 - val_auc: 0.0000e+00
Epoch 105/200
7/7 [==============================] - 6s 885ms/step - loss: 0.0680 - accuracy: 0.5469 - auc: 0.0000e+00 - val_loss: 0.0567 - val_accuracy: 0.5882 - val_auc: 0.0000e+00
Epoch 106/200
7/7 [==============================] - 6s 888ms/step - loss: 0.0678 - a

Epoch 146/200
7/7 [==============================] - 6s 891ms/step - loss: 0.0668 - accuracy: 0.5156 - auc: 0.0000e+00 - val_loss: 0.0567 - val_accuracy: 0.4706 - val_auc: 0.0000e+00
Epoch 147/200
7/7 [==============================] - 6s 915ms/step - loss: 0.0668 - accuracy: 0.5312 - auc: 0.0000e+00 - val_loss: 0.0567 - val_accuracy: 0.5294 - val_auc: 0.0000e+00
Epoch 148/200
7/7 [==============================] - 6s 911ms/step - loss: 0.0668 - accuracy: 0.5625 - auc: 0.0000e+00 - val_loss: 0.0567 - val_accuracy: 0.5294 - val_auc: 0.0000e+00
Epoch 149/200
7/7 [==============================] - 6s 943ms/step - loss: 0.0668 - accuracy: 0.5312 - auc: 0.0000e+00 - val_loss: 0.0567 - val_accuracy: 0.5294 - val_auc: 0.0000e+00
Epoch 150/200
7/7 [==============================] - 6s 939ms/step - loss: 0.0668 - accuracy: 0.5469 - auc: 0.0000e+00 - val_loss: 0.0567 - val_accuracy: 0.5294 - val_auc: 0.0000e+00
Epoch 151/200
7/7 [==============================] - 6s 924ms/step - loss: 0.0668 - a

Epoch 191/200
7/7 [==============================] - 6s 932ms/step - loss: 0.0680 - accuracy: 0.5625 - auc: 0.0000e+00 - val_loss: 0.0586 - val_accuracy: 0.4706 - val_auc: 0.0000e+00
Epoch 192/200
7/7 [==============================] - 6s 915ms/step - loss: 0.0677 - accuracy: 0.5469 - auc: 0.0000e+00 - val_loss: 0.0571 - val_accuracy: 0.4118 - val_auc: 0.0000e+00
Epoch 193/200
7/7 [==============================] - 6s 888ms/step - loss: 0.0679 - accuracy: 0.5938 - auc: 0.0000e+00 - val_loss: 0.0579 - val_accuracy: 0.4706 - val_auc: 0.0000e+00
Epoch 194/200
7/7 [==============================] - 6s 882ms/step - loss: 0.0681 - accuracy: 0.5781 - auc: 0.0000e+00 - val_loss: 0.0575 - val_accuracy: 0.4706 - val_auc: 0.0000e+00
Epoch 195/200
7/7 [==============================] - 6s 875ms/step - loss: 0.0678 - accuracy: 0.5469 - auc: 0.0000e+00 - val_loss: 0.0571 - val_accuracy: 0.4706 - val_auc: 0.0000e+00
Epoch 196/200
7/7 [==============================] - 6s 885ms/step - loss: 0.0676 - a

In [109]:
model.save('./models/object_detection_2.h5') #lưu lại mô hình